In [1]:
from gensim.models import KeyedVectors
import pandas as pd
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

In [2]:
model = KeyedVectors.load("300features_40minwords_10context")
num_features = 300

In [7]:
model.wv.syn0.shape

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


(1150, 300)

In [8]:
import pandas as pd
train = pd.read_csv("./datasets/labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)
test = pd.read_csv("./datasets/testData.tsv", header=0, delimiter="\t", quoting=3)
unlabeled_train = pd.read_csv("./datasets/unlabeledTrainData.tsv", header=0, delimiter="\t", quoting=3)

In [9]:
def review_to_wordlist(review, remove_stopwords=False):
    review_text = BeautifulSoup(review).get_text()
    letters_only = re.sub("[^a-zA-Z]", " ", review_text)
    
    words = letters_only.lower().split()
    if remove_stopwords:
        stops = set(stopwords.words('english'))
        words = [w for w in words if not w in stops]
    return words 

In [10]:
import numpy as np
def makeFeatureVec(words, model, num_features):
    featureVec = np.zeros((num_features), dtype='float32')
    nwords = 0
    
    index2word_set = set(model.wv.index2word)
    
    for word in words:
        if word in index2word_set:
            nwords += 1
            featureVec = np.add(featureVec, model[word])
    featureVec = np.divide(featureVec, nwords)
    return featureVec

In [11]:
def getAvgFeatureVecs(reviews, model, num_features):
    counter = 0
    reviewFeatureVecs = np.zeros((len(reviews), num_features), dtype='float32')
    for review in reviews:
        if counter % 1000 == 0:
            print("Review %d of %d" % (counter, len(reviews)))
        reviewFeatureVecs[counter] = makeFeatureVec(review, model, num_features)
        counter += 1
    return reviewFeatureVecs

In [12]:
clean_train_reviews = []
number = 1000
for review in train["review"][:number]:
    clean_train_reviews.append(review_to_wordlist(review, remove_stopwords=True))
trainDataVecs = getAvgFeatureVecs(clean_train_reviews, model, num_features)
print("Creating average feature vecs for text reviews")
clean_test_reviews = []
for review in test["review"][:number]:
    clean_test_reviews.append(review_to_wordlist(review, remove_stopwords=True))
testDataVecs = getAvgFeatureVecs(clean_test_reviews, model, num_features)

/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


Review 0 of 1000


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


Creating average feature vecs for text reviews
Review 0 of 1000


In [78]:
# # Fit a random forest to the training data, using 100 trees
from sklearn.ensemble import RandomForestClassifier
# forest = RandomForestClassifier( n_estimators = 100 )

# print ("Fitting a random forest to labeled training data...")
# forest = forest.fit( trainDataVecs, train["sentiment"] )

# # Test & extract results 
# result = forest.predict( testDataVecs )

# # Write the test results 
# output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
# output.to_csv( "result_03.csv", index=False, quoting=3 )

In [13]:
from sklearn.cluster import KMeans
import time


In [15]:
start = time.time()
word_features = model.wv.syn0
num_clusters =int( word_features.shape[0] / 5)
kmeans_clustering = KMeans(n_clusters=int(num_clusters))
idx = kmeans_clustering.fit_predict(word_features)
end = time.time()
elapsed = end - start
print("Time taken for K Means clustering: ", elapsed, "seconds.")

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  


Time taken for K Means clustering:  2.264671802520752 seconds.


In [16]:
# For the first 10 clusters
word_centroid_map = dict(zip( model.wv.index2word, idx ))

for cluster in range(0,10):
#     #
#     # Print the cluster number  
    print("Cluster %d" % cluster)
#     #
#     # Find all of the words for that cluster number, and print them out
    words = []
    for i in range(0,len(word_centroid_map.values())):
        if( list(word_centroid_map.values())[i] == cluster ):
            words.append(list(word_centroid_map.keys())[i])
    print(words)

Cluster 0
['second']
Cluster 1
['slow', 'hilarious', 'particularly', 'soundtrack', 'ridiculous', 'cute', 'believable', 'flat', 'exciting', 'odd', 'acted', 'awesome']
Cluster 2
['new', 'school', 'death', 'small', 'town', 'whose']
Cluster 3
['saying', 'avoid', 'review', 'comments', 'reviews', 'sorry', 'comment', 'expecting']
Cluster 4
['yet', 'looking', 'stay', 'talk', 'begin', 'question', 'whatever', 'joke']
Cluster 5
['getting', 'lost', 'past', 'soon', 'coming', 'power', 'turned', 'trouble', 'changed']
Cluster 6
['fact', 'truly', 'classic', 'writing']
Cluster 7
['these']
Cluster 8
['shot', 'perfect', 'lead', 'michael', 'moves']
Cluster 9
['show', 'hope', 'able', 'wanted']


In [17]:
def create_bag_of_centroids(wordlist, word_centroid_map):
    num_centroids = max(list(word_centroid_map.values())) + 1
    bag_of_centroids = np.zeros(num_centroids, dtype='float32')
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    return bag_of_centroids

In [18]:
train_cetroids = np.zeros((train["review"][:number].size, num_clusters), dtype="float32")

In [19]:
counter = 0
for review in clean_train_reviews:
    train_cetroids[counter] = create_bag_of_centroids(review, word_centroid_map)
    counter += 1

In [20]:
test_cetroids = np.zeros((test["review"][:number].size, num_clusters), dtype="float32")
counter = 0
for review in clean_test_reviews:
    test_cetroids[counter] = create_bag_of_centroids(review, word_centroid_map)
    counter += 1

In [22]:
# Fit a random forest and extract predictions 
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators = 100)

# Fitting the forest may take a few minutes
print("Fitting a random forest to labeled training data...")
forest = forest.fit(train_cetroids,train["sentiment"][:1000])
# result = forest.predict(test_cetroids)

# Write the test results 
# output = pd.DataFrame(data={"id":test["id"], "sentiment":result})
# output.to_csv( "result_03_2.csv", index=False, quoting=3 )

Fitting a random forest to labeled training data...
